In [ ]:
import tensorflow as tf
! pip install tensorflow_datasets
import tensorflow_datasets as tfds

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
tfds.list_builders()

In [ ]:
(ds_train, ds_test), info = tfds.load(
    'cats_vs_dogs',
    split=['train[:70%]', 'train[70%:]'],
    with_info=True,
    as_supervised=True,
)

In [ ]:
print(info)

In [ ]:
print("Training Set Size: %d" % ds_train.cardinality().numpy())
print("Test Set Size: %d" % ds_test.cardinality().numpy())

In [ ]:
num_classes = info.features['label'].num_classes
print("Number of Classes: %d" % num_classes)



In [ ]:
ctr = 0
plt.rcParams["figure.figsize"] = [30, 15]
plt.rcParams["figure.autolayout"] = True
  
for image, label in ds_train:
    image = image.numpy()
    print(image.shape[1])
    plt.subplot(1, 5, ctr+1)
    plt.title('Label {}'.format(label))
    plt.imshow(image, cmap=plt.cm.binary)
    ctr += 1
    if ctr == 5:
        break
  
plt.show()

In [ ]:
for i, example in enumerate(ds_train.take(10)):
    shape = example[0].shape
    print("Image %d -> shape: (%d, %d) label: %d" %
          (i, shape[0], shape[1], example[1]))



In [ ]:
min_shape = (1000,1000)
max_shape = (0,0)
add_shape = [0,0]
cnt = 0

ds_clean = []


for i, example in enumerate(ds_train):
    cnt = cnt +1
    shape = example[0].shape
    min_shape_area = min_shape[0]*min_shape[1]
    area = shape[0] * shape[1]
    if area < min_shape_area:
        min_shape = shape
    max_shape_area = max_shape[0]*max_shape[1]
    if area > max_shape_area:
        max_shape = shape
        
        
    add_shape[0] = add_shape[0] + shape[0]
    add_shape[1] = add_shape[1] + shape[1]
    
    ds_clean.append(example)
          
        
        
        
add_shape[0] = add_shape[0] / cnt
add_shape[1] = add_shape[1] / cnt
  
       
print("Image %d -> shape: (%d, %d) " %
        (i, min_shape[0], min_shape[1] ))
print("Image %d -> shape: (%d, %d) " %
        (i, max_shape[0], max_shape[1] ))
print(add_shape)

print(cnt)

In [41]:
IMG_SIZE = 100
IMG_WIDTH = 224
IMG_HEIGHT = 224

In [42]:
def format_image(image, label):
  
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
      
    # Normalisation
    image = image/255.0
    return image, label
  
batch_size = 32
ds_train = ds_train.shuffle(300).map(
    format_image).batch(batch_size).prefetch(1)
ds_test = ds_test.map(format_image).batch(batch_size).prefetch(1)

ValueError: in user code:

    File "/tmp/ipykernel_20997/3101195762.py", line 3, in format_image  *
        image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))

    ValueError: 'images' must have either 3 or 4 dimensions.


In [31]:
print(ds_train)
print(ds_test)

<PrefetchDataset element_spec=(TensorSpec(shape=(None, None, 100, 100, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, None), dtype=tf.int64, name=None))>
<PrefetchDataset element_spec=(TensorSpec(shape=(None, None, 100, 100, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, None), dtype=tf.int64, name=None))>


In [29]:

def getModel(image_shape):
    mobileNet = tf.keras.applications.mobilenet.MobileNet(image_shape)
    X = mobileNet.layers[-2].output
    X_output = tf.keras.layers.Dense(1,
                                     activation='relu')(X)
    model = tf.keras.models.Model(inputs=mobileNet.input,
                                  outputs=X_output)
    return model
  
model = getModel((IMG_SIZE, IMG_SIZE, 3))
  
optimizer = tf.keras.optimizers.Adam()
loss = 'mean_squared_error'
model.compile(optimizer=optimizer, 
              loss=loss,
              metrics='accuracy')
  
epochs = 5
model.fit(ds_train, epochs=epochs, 
          validation_data=ds_test)

ValueError: When setting `include_top=True` and loading `imagenet` weights, `input_shape` should be (224, 224, 3).  Received: input_shape=(100, 100, 3)